# SPARK assignment

### Patricia Llull Sperandio

The dataset is related to crime in Chicago for a specific time frame (2005 to 2007).

The analysis follows this structure:
1. PySpark environment setup
2. Data source and Spark data abstraction setup
3. Data set metadata analysis
    1. Schema and size of dataframe
    2. Random sample(s)
    3. Data entities, metrics, dimensions
    4. Column categorisation (organisation of fields)
        1. Changes in Date Column (split date)
        2. Getting day of week and hour (24h)
        3. Location columns: data types and renaming
4. Basic profiling of column groups
    1. **Location related** profiling
    2. **Time-related** profiling
    3. **Crime-specific** profiling
    
5. Insights
    1. Top 5's
    2. Ratio of number of crimes & frequency of crime type by day time category
    3. Ratio of crimes types: week vs. weekend
    4. Safety in the districts

## 1. PySpark environment setup

In [1]:
#ENVIRONMENT
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sc.version
sc.pythonVer
sc.master
print(f"For the analysis I will be using SparkContext version {sc.version} and Spark will be using version {sc.pythonVer} of Python.")

For the analysis I will be using SparkContext version 2.4.4 and Spark will be using version 3.7 of Python.


## 2. Data source and Spark data abstraction setup (DataFrame)

Data source: Kaggle.
This is the <a href="https://www.kaggle.com/currie32/crimes-in-chicago" title="Kaggle"> link</a> to the dataset.

Spark abstraction: convert the csv into a dataframe i.e create a representation of our data using Spark structures. I use the option infer the schema: Spark will determine data types in the columns; option header true which means the first row is column description/metadata; and csv because we are reading a csv file.




In [2]:
#LOAD DATA
chicagoDF = spark.read\
                        .option("inferSchema", "true")\
                        .option("header","true")\
                        .csv("Chicago_Crimes_2005_to_2007.csv") 

## 3. Dataset and metadata analysis
> ### A. Display schema and size of dataframe

In [3]:
#DATA FRAME STRUCTURE - rows and columns & object type
from IPython.display import display, Markdown
display(Markdown("#### Column data types"))

display(Markdown("Each column has a specific data type:"))
chicagoDF.printSchema()

display(Markdown("#### Dimensions of the data frame"))
display(Markdown("This dataframe has **%d rows**." % chicagoDF.count()))
a = len(chicagoDF.columns)
display(Markdown(f"This dataframe has **{a} columns**."))
b = type(chicagoDF)
display(Markdown(f"This class of object is **{b}** "))

display(Markdown("Spark infers the schema, and it has recognised some numeric columns straight away. However, \
it might be necessary to make some changes in order to perform calculations. For example, the date column will\
be split in order to generate insights."))

#### Column data types

Each column has a specific data type:

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: double (nullable = true)
 |-- Ward: double (nullable = true)
 |-- Community Area: double (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: double (nullable = true)
 |-- Y Coordinate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



#### Dimensions of the data frame

This dataframe has **1872345 rows**.

This dataframe has **23 columns**.

This class of object is **<class 'pyspark.sql.dataframe.DataFrame'>** 

Spark infers the schema, and it has recognised some numeric columns straight away. However, it might be necessary to make some changes in order to perform calculations. For example, the date column willbe split in order to generate insights.

#### Columns Description (<a href="https://www.kaggle.com/currie32/crimes-in-chicago" title="Kaggle">source</a>) <br>  

1. **ID** - Unique identifier for the record.
2. **Case Number** - The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.
3. **Date** - Date when the incident occurred. this is sometimes a best estimate.
4. **Block** - The partially redacted address where the incident occurred, placing it on the same block as the actual address.
5. **IUCR** - The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.
6. **Primary Type** - The primary description of the IUCR code.
7. **Description** - The secondary description of the IUCR code, a subcategory of the primary description.
8. **Location Description** - Description of the location where the incident occurred.

9. **Arrest** - Indicates whether an arrest was made.

10. **Domestic** - Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.

11. **Beat** - Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74.

12. **District** - Indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r.

13. **Ward** - The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76.

14. **Community Area** - Indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6.

15. **FBI Code** - Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html.

16. **X Coordinate** - The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.

17. **Y Coordinate** - The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.

18. **Year** - Year the incident occurred.

19. **Updated On** - Date and time the record was last updated.

20. **Latitude** - The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.

21. **Longitude** - The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.

22. **Location** - The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.



In [4]:
from IPython.display import display, Markdown

display(Markdown(">### B.Sample the dataset"))

display(Markdown("**Example of a random sample** usning 20% of the dataset. Here showing 3 rows of that sample:"))

print(chicagoDF.sample(False, 0.2).take(3))

>### B.Sample the dataset

**Example of a random sample** usning 20% of the dataset. Here showing 3 rows of that sample:

[Row(_c0=8, ID=4673633, Case Number='HM275105', Date='04/05/2006 03:00:00 PM', Block='067XX S ROCKWELL ST', IUCR='0820', Primary Type='THEFT', Description='$500 AND UNDER', Location Description='STREET', Arrest=False, Domestic=False, Beat=832, District=8.0, Ward=15.0, Community Area=66.0, FBI Code='06', X Coordinate=1160205.0, Y Coordinate=1859776.0, Year=2006, Updated On='04/15/2016 08:55:02 AM', Latitude=41.770925978, Longitude=-87.688304107, Location='(41.770925978, -87.688304107)'), Row(_c0=14, ID=4673642, Case Number='HM202299', Date='02/26/2006 02:47:21 PM', Block='002XX S LEAMINGTON AVE', IUCR='1811', Primary Type='NARCOTICS', Description='POSS: CANNABIS 30GMS OR LESS', Location Description='SIDEWALK', Arrest=True, Domestic=False, Beat=1533, District=15.0, Ward=28.0, Community Area=25.0, FBI Code='18', X Coordinate=1142168.0, Y Coordinate=1898610.0, Year=2006, Updated On='04/15/2016 08:55:02 AM', Latitude=41.87784456, Longitude=-87.753461293, Location='(41.87784456, -87.75346129

In [5]:
#SAMPLE FOR the analyis (10% of the data)  ; run only once otherwise it takes a sample of the sample

display(Markdown("A sample of dataframe ChicagoDF will be used to perform operations. It will be cached \
to make operations faster later on.  I use a sample because the dataset contains over a million rows.<br>\
        <br>The sample  contains 10% of the data. Sampling method: without replacement. <br>."))
chicagoDF = chicagoDF.sample(False, 0.1)
chicagoDF.count() #make sure the sample is +100K rows (should be the case as I take 10% of +1M)

A sample of dataframe ChicagoDF will be used to perform operations. It will be cached to make operations faster later on.  I use a sample because the dataset contains over a million rows.<br>        <br>The sample  contains 10% of the data. Sampling method: without replacement. <br>.

186916

In [6]:
chicagoDF.cache() #optimise the process (make it faster)

DataFrame[_c0: int, ID: int, Case Number: string, Date: string, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: boolean, Domestic: boolean, Beat: int, District: double, Ward: double, Community Area: double, FBI Code: string, X Coordinate: double, Y Coordinate: double, Year: int, Updated On: string, Latitude: double, Longitude: double, Location: string]

>### C. Data entities, metrics and dimensions

The dataframe contains the following elements
* **Entities**: Crime, Location, Date <br>
* **Metrics**: Time of the crime, the violence of the crime through several attributes such as primary type <br>
* **Dimensions**: Day, Description, District, Block, ...

In [7]:
#Entities, dimensions and metrics

from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit, split, dayofweek
from pyspark.sql.types import IntegerType

#SPLIT DATE, ADD DayOfWeek, AM_PM
display(Markdown(">#### 1. Date column: splitting to get Month, DayOfMonth, Hour, AM_PM"))

display(Markdown("I wil split the date column to generate insights in the analysis later on.\
 I cast Day and Month as integers."))

#time of day stays an integer because of its format.
display(Markdown("This is the resulting schema for the dataframe:"))
chicagoDF= chicagoDF.withColumn("Month", split(col("Date"), "/").getItem(0).cast(IntegerType()))\
.withColumn("DayOfMonth", split(col("Date"), "/").getItem(1).cast(IntegerType()))\
.withColumn("TimeOfDay", split(col("Date"), " ").getItem(1))\
.withColumn("Hour", split(col("TimeOfDay"), ":").getItem(0).cast(IntegerType()))\
.withColumn("AM_PM", split(col("Date")," ").getItem(2))

#new SCHEMA with data split and hours too
#chicagoDF.printSchema()


#Check the time-related columns
display(Markdown("Check the resulting structure of time-related columns"))
chicagoDF.select("Date","TimeOfDay","DayOfMonth", "Month","Hour", "AM_PM").limit(10).show()

>#### 1. Date column: splitting to get Month, DayOfMonth, Hour, AM_PM

I wil split the date column to generate insights in the analysis later on. I cast Day and Month as integers.

This is the resulting schema for the dataframe:

Check the resulting structure of time-related columns

+--------------------+---------+----------+-----+----+-----+
|                Date|TimeOfDay|DayOfMonth|Month|Hour|AM_PM|
+--------------------+---------+----------+-----+----+-----+
|03/31/2006 08:20:...| 08:20:00|        31|    3|   8|   AM|
|04/05/2006 05:00:...| 05:00:00|         5|    4|   5|   PM|
|04/06/2006 01:00:...| 01:00:00|         6|    4|   1|   AM|
|04/02/2006 10:00:...| 10:00:00|         2|    4|  10|   AM|
|04/01/2006 12:00:...| 12:00:00|         1|    4|  12|   AM|
|04/03/2006 09:00:...| 09:00:00|         3|    4|   9|   PM|
|03/01/2006 10:09:...| 10:09:19|         1|    3|  10|   PM|
|02/16/2006 08:38:...| 08:38:53|        16|    2|   8|   PM|
|04/03/2006 05:00:...| 05:00:00|         3|    4|   5|   PM|
|03/01/2006 02:30:...| 02:30:00|         1|    3|   2|   PM|
+--------------------+---------+----------+-----+----+-----+



In [8]:
#ADDING DAY OF WEEK

display(Markdown(">#### 2. Date format transformations: getting DayOfWeek and exact hour of the day (24h format)"))


#import functions
from pyspark.sql.functions import unix_timestamp, from_unixtime, year, month, dayofmonth, dayofweek, date_format
from pyspark.sql.functions import concat, lit
import pyspark.sql.functions as F 

#Add columns for: NEW DATE FORMAT - DAY OF WEEK -  hour_am_pm


chicagoDF= chicagoDF.withColumn("Date_new",from_unixtime(unix_timestamp('Date', 'MM/dd/yyy')))\
.withColumn('DayOfWeek', date_format(col('Date_new'), 'u').cast(IntegerType()))\
.withColumn('DayOfWeek_str', date_format(col('Date_new'), 'E'))\
.withColumn('hour_am_pm', concat(col('Hour'),lit(' '), col('AM_PM')))\
.withColumn("hour_am_pm", F.when(F.col("hour_am_pm") == '1 PM', 13)\
                                                           .when(F.col("hour_am_pm") == '2 PM', 14)\
                                                           .when(F.col("hour_am_pm") == '3 PM', 15)\
                                                           .when(F.col("hour_am_pm") == '4 PM', 16)\
                                                           .when(F.col("hour_am_pm") == '5 PM', 17)\
                                                           .when(F.col("hour_am_pm") == '6 PM', 18)\
                                                           .when(F.col("hour_am_pm") == '7 PM', 19)\
                                                           .when(F.col("hour_am_pm") == '8 PM', 20)\
                                                           .when(F.col("hour_am_pm") == '9 PM',  21)\
                                                           .when(F.col("hour_am_pm") == '10 PM', 22)\
                                                           .when(F.col("hour_am_pm") == '11 PM', 23)\
                                                           .when(F.col("hour_am_pm") == '12 PM', 12)\
                                                           .when(F.col("hour_am_pm") == '1 AM', 1)\
                                                           .when(F.col("hour_am_pm") == '2 AM', 2)\
                                                           .when(F.col("hour_am_pm") == '3 AM', 3)\
                                                           .when(F.col("hour_am_pm") == '4 AM', 4)\
                                                           .when(F.col("hour_am_pm") == '5 AM', 5)\
                                                           .when(F.col("hour_am_pm") == '6 AM', 6)\
                                                           .when(F.col("hour_am_pm") == '7 AM', 7)\
                                                           .when(F.col("hour_am_pm") == '8 AM', 8)\
                                                           .when(F.col("hour_am_pm") == '9 AM', 9)\
                                                           .when(F.col("hour_am_pm") == '10 AM', 10)\
                                                           .when(F.col("hour_am_pm") == '11 AM', 11)\
                                                           .when(F.col("hour_am_pm") == '12 AM', 24))\

                                                            
                                                          
chicagoDF.select("Date","DayOfWeek","DayOfWeek_str", "Hour", "AM_PM", "hour_am_pm").limit(5).show()

>#### 2. Date format transformations: getting DayOfWeek and exact hour of the day (24h format)

+--------------------+---------+-------------+----+-----+----------+
|                Date|DayOfWeek|DayOfWeek_str|Hour|AM_PM|hour_am_pm|
+--------------------+---------+-------------+----+-----+----------+
|03/31/2006 08:20:...|        5|          Fri|   8|   AM|         8|
|04/05/2006 05:00:...|        3|          Wed|   5|   PM|        17|
|04/06/2006 01:00:...|        4|          Thu|   1|   AM|         1|
|04/02/2006 10:00:...|        7|          Sun|  10|   AM|        10|
|04/01/2006 12:00:...|        6|          Sat|  12|   AM|        24|
+--------------------+---------+-------------+----+-----+----------+



In [9]:
#CHANGING DATA TYPES of locations
display(Markdown(">#### 3. Location columns: data type changes"))

#Change data type for District, Ward, Community Area (and rename Community Area to Community; and Location Description to LocationDescription)
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit, split

chicagoDF = chicagoDF.withColumn("District",col("District").cast(IntegerType()))\
.withColumn("Ward",col("Ward").cast(IntegerType()))\
.withColumn("Community Area", col("Community Area").cast(IntegerType()))\
.withColumnRenamed("Community Area", "Community")\
.withColumnRenamed("Location Description", "LocationDescription")


>#### 3. Location columns: data type changes

In [10]:
#print the new schema and check the data type of day of week is integer
display(Markdown("This is the new schema after making the changes in date columns, date format and locations columns: adding date related columns, changing some data types and column names"))
chicagoDF.printSchema()

This is the new schema after making the changes in date columns, date format and locations columns: adding date related columns, changing some data types and column names

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: double (nullable = true)
 |-- Y Coordinate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayOfMonth: integer (nulla

> ### D. Column Categorisation

Potential groups of columns:
- **Identification-related**: *ID, Case number, FBI code* 
- **Time-related**: *Date, Year, Month, DayOfMonth, DayOfWeek (string), DayOfWeek(integer), TimeOfDay, Hour, Hour_am_pm*
- **Location-related**:*Location description, Beat, District, Ward, Community Area, X coordinate, Y coordinate, Latitude, Longitude, Location*
- **Type of offense**: *IUCR, Primary Type, Description, Arrest, Domestic*


## 4. Basic profiling of column groups

Deep-dive into the column categories related to time, location and type of offense.

In [11]:
display(Markdown(">### A. Location-related columns profiling"))
display(Markdown("Columns of interest: Block, LocationDescription, Beat, District, Ward, Community, X Coor, Y coord, Lon, Lat, Location"))
display(Markdown("Location related columns are important for crime data: they give insights as to where crimes are committed and how often.\
i.e. which districts / communities are safer; which are unsafe."))

#1. CHECK NA's : do it separately for strings vs. integer columns
#Some location idenfiers have 9K missing values which is still little compared to 1M dataset.
import pyspark.sql.functions as F
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit, split
from pyspark.sql.types import IntegerType

display(Markdown("Check for null in location-related columns:"))
chicagoDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Beat","Ward","Community"]]).show()

chicagoDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Block","District","LocationDescription", "Location"]]).show()

chicagoDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Longitude","Latitude","X Coordinate","Y Coordinate"]]).show()

display(Markdown("Because the dataset contains over a million values, I will remove the rows that contain null values."))


#2. CLEAN THE DATASET, remove NA's
chicagoDF= chicagoDF.where("LocationDescription is not null and District is not null and Ward is not \
null and Community is not null and Longitude is not null" ).cache()

#3. Verify no more NA's
display(Markdown("In the new dataframe cleanchicagoDF, check that there are no more missing values in location columns:"))

chicagoDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Beat","Ward","Community"]]).show()

chicagoDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Block","District","LocationDescription","Location"]]).show()

chicagoDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Longitude","Latitude","X Coordinate", "Y coordinate"]]).show()

>### A. Location-related columns profiling

Columns of interest: Block, LocationDescription, Beat, District, Ward, Community, X Coor, Y coord, Lon, Lat, Location

Location related columns are important for crime data: they give insights as to where crimes are committed and how often.i.e. which districts / communities are safer; which are unsafe.

Check for null in location-related columns:

+----+----+---------+
|Beat|Ward|Community|
+----+----+---------+
|   0|   1|       38|
+----+----+---------+

+-----+--------+-------------------+--------+
|Block|District|LocationDescription|Location|
+-----+--------+-------------------+--------+
|    0|       0|                  4|     918|
+-----+--------+-------------------+--------+

+---------+--------+------------+------------+
|Longitude|Latitude|X Coordinate|Y Coordinate|
+---------+--------+------------+------------+
|      918|     918|         918|         918|
+---------+--------+------------+------------+



Because the dataset contains over a million values, I will remove the rows that contain null values.

In the new dataframe cleanchicagoDF, check that there are no more missing values in location columns:

+----+----+---------+
|Beat|Ward|Community|
+----+----+---------+
|   0|   0|        0|
+----+----+---------+

+-----+--------+-------------------+--------+
|Block|District|LocationDescription|Location|
+-----+--------+-------------------+--------+
|    0|       0|                  0|       0|
+-----+--------+-------------------+--------+

+---------+--------+------------+------------+
|Longitude|Latitude|X Coordinate|Y coordinate|
+---------+--------+------------+------------+
|        0|       0|           0|           0|
+---------+--------+------------+------------+



In [12]:
#4. NOW THAT THE DATASET CONTAINS NO NA's , look at distinct values, least vs. most freq
display(Markdown("Now that the dataset is clean, investigate location columns:"))

BlockDF     = chicagoDF.groupBy("Block").agg(count(lit(1)).alias("Total"))
LocDesDF    = chicagoDF.groupBy("LocationDescription").agg(count(lit(1)).alias("Total"))
BeatDF      = chicagoDF.groupBy("Beat").agg(count(lit(1)).alias("Total"))
DistrictDF  = chicagoDF.groupBy("District").agg(count(lit(1)).alias("Total"))
LocationDF  = chicagoDF.groupBy("Location").agg(count(lit(1)).alias("Total"))

display(Markdown("Check number of distinct values in columns Block, Location Description, Beat, District,\nWard, Community and Location"))

chicagoDF.select([countDistinct(c).alias(c) for c in ["Block","LocationDescription","Beat","District","Ward","Community", "Location"]]).show()
    
display(Markdown("This are  **%d** different block locations." % BlockDF.count()))
display(Markdown("This are  **%d** different location description." % LocDesDF.count()))
display(Markdown("This are  **%d** different districts." % DistrictDF.count()))
display(Markdown("This are  **%d** different locations." % LocationDF.count()))

display(Markdown("Most and least frequent occurrences for Location columns:"))


leastFreqBlock    = BlockDF.orderBy(col("Total").asc()).first()
mostFreqBlock     = BlockDF.orderBy(col("Total").desc()).first()
leastFreqLocDes   = LocDesDF.orderBy(col("Total").asc()).first()
mostFreqLocDes    = LocDesDF.orderBy(col("Total").desc()).first()
leastFreqBeat     = BeatDF.orderBy(col("Total").asc()).first()
mostFreqBeat      = BeatDF.orderBy(col("Total").desc()).first()
leastFreqDistrict = DistrictDF.orderBy(col("Total").asc()).first()
mostFreqDistrict  = DistrictDF.orderBy(col("Total").desc()).first()
leastFreqLocation = LocationDF.orderBy(col("Total").asc()).first()
mostFreqLocation  = LocationDF.orderBy(col("Total").desc()).first()


#Attention dans la partie du bas """ % (..)
# le "%d" or "%s" dépend du data type (digits ou strings!!!)
display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqBeat", "mostFreqBeat","leastFreqDistrict", "mostFreqDistrict", \
       "%d (%d occurrences)" % (leastFreqBeat["Beat"], leastFreqBeat["Total"]), \
       "%d (%d occurrences)" % (mostFreqBeat["Beat"], mostFreqBeat["Total"]), \
       "%d (%d occurrences)" % (leastFreqDistrict["District"], leastFreqDistrict["Total"]), \
       "%d (%d occurrences)" % (mostFreqDistrict["District"], mostFreqDistrict["Total"]))))

display(Markdown("""
| %s | %s |
|----|----|
| %s | %s |
""" % ("leastFreqBlock", "mostFreqBlock", \
       "%s (%d occurrences)" % (leastFreqBlock["Block"], leastFreqBlock["Total"]), \
       "%s (%d occurrences)" % (mostFreqBlock["Block"], mostFreqBlock["Total"]))))

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqLocDes", "mostFreqLocDes", "leastFreqLocation","mostFreqLocation", \
       "%s (%d occurrences)" % (leastFreqLocDes["LocationDescription"], leastFreqLocDes["Total"]), \
       "%s (%d occurrences)" % (mostFreqLocDes["LocationDescription"], mostFreqLocDes["Total"]),\
       "%s (%d occurrences)" % (leastFreqLocation["Location"], leastFreqLocation["Total"]), \
       "%s (%d occurrences)" % (mostFreqLocation["Location"], leastFreqLocation["Total"]))))


Now that the dataset is clean, investigate location columns:

Check number of distinct values in columns Block, Location Description, Beat, District,
Ward, Community and Location

+-----+-------------------+----+--------+----+---------+--------+
|Block|LocationDescription|Beat|District|Ward|Community|Location|
+-----+-------------------+----+--------+----+---------+--------+
|25605|                 97| 284|      23|  50|       78|  107436|
+-----+-------------------+----+--------+----+---------+--------+



This are  **25605** different block locations.

This are  **97** different location description.

...

This are  **107436** different locations.

Most and least frequent occurrences for Location columns:


| leastFreqBeat | mostFreqBeat | leastFreqDistrict | mostFreqDistrict |
|----|----|----|----|
| 1653 (1 occurrences) | 423 (1580 occurrences) | 31 (2 occurrences) | 8 (13339 occurrences) |



| leastFreqBlock | mostFreqBlock |
|----|----|
| 060XX N MAPLEWOOD AVE (1 occurrences) | 100XX W OHARE ST (682 occurrences) |



| leastFreqLocDes | mostFreqLocDes | leastFreqLocation | mostFreqLocation |
|----|----|----|----|
| BASEMENT (1 occurrences) | STREET (50036 occurrences) | (41.774946007, -87.764049247) (1 occurrences) | (41.976290414, -87.905227221) (1 occurrences) |


In [13]:
display(Markdown(">### B. Time-related columns profiling"))
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit

#Get a general idea of the distribution of the data
display(Markdown("Summary of columns Year, Month, DayOfMonth, DayOfWeek, Hour and hour_am_pm:"))
chicagoDF.select("Year","Month","DayOfMonth", "DayOfWeek","Hour","hour_am_pm").summary().show()

#check for nulls to make sure we can perform the calculations needed for the analysis
display(Markdown("Check for nulls on columns Year, Month, Day of month, and Day of week, Hour and hour_am_pm:"))
chicagoDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Year", "Month", "DayOfMonth","DayOfWeek","Hour", "hour_am_pm"]]).show()
display(Markdown("There are no missing values, which is essential for further analysis."))

#check the number of distinct values to make sure the date columns have values consistent with the calendar
display(Markdown("Check the amount of distinct values in columns Year, Month, DayOfMonth, DayOfWeek, Hour and hour_am_pm :"))
chicagoDF.select([countDistinct(c).alias(c) for c in ["Year", "Month", "DayOfMonth", "DayOfWeek", "Hour","hour_am_pm"]]).show()

>### B. Time-related columns profiling

Summary of columns Year, Month, DayOfMonth, DayOfWeek, Hour and hour_am_pm:

+-------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|summary|              Year|            Month|        DayOfMonth|         DayOfWeek|             Hour|        hour_am_pm|
+-------+------------------+-----------------+------------------+------------------+-----------------+------------------+
|  count|            185959|           185959|            185959|            185959|           185959|            185959|
|   mean|2006.0914610209777| 6.46829139756613|15.638630020595938|3.9937835759495375|6.972977914486527|14.589753655375647|
| stddev|0.7518871419099914|3.279803445193015|  8.82818594774572| 1.975329964010372|3.555519838825136|  6.45521954796434|
|    min|              2005|                1|                 1|                 1|                1|                 1|
|    25%|              2006|                4|                 8|                 2|                4|                10|
|    50%|              2

Check for nulls on columns Year, Month, Day of month, and Day of week, Hour and hour_am_pm:

+----+-----+----------+---------+----+----------+
|Year|Month|DayOfMonth|DayOfWeek|Hour|hour_am_pm|
+----+-----+----------+---------+----+----------+
|   0|    0|         0|        0|   0|         0|
+----+-----+----------+---------+----+----------+



There are no missing values, which is essential for further analysis.

Check the amount of distinct values in columns Year, Month, DayOfMonth, DayOfWeek, Hour and hour_am_pm :

+----+-----+----------+---------+----+----------+
|Year|Month|DayOfMonth|DayOfWeek|Hour|hour_am_pm|
+----+-----+----------+---------+----+----------+
|   3|   12|        31|        7|  12|        24|
+----+-----+----------+---------+----+----------+



In [14]:
#Look at extremes for time-related values

#Look at extremes: hours am/pm
display(Markdown("Most and least frequent occurrences for hour_am_pm:"))
HourOccurrencesDF = chicagoDF.groupBy("hour_am_pm").agg(count(lit(1)).alias("Total"))

leastFreqHour    = HourOccurrencesDF.orderBy(col("Total").asc()).first()
mostFreqHour     = HourOccurrencesDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s |
|----|----|
| %s | %s |
""" % ("leastFreqHour", "mostFreqHour", \
       "%d (%d occurrences)" % (leastFreqHour["hour_am_pm"], leastFreqHour["Total"]), \
       "%d (%d occurrences)" % (mostFreqHour["hour_am_pm"], mostFreqHour["Total"]))))
display(Markdown(f"The most frequent hour is  {(mostFreqHour['hour_am_pm']) }"))


#Look at extremes for day of month & day of week
display(Markdown("Most and least frequent occurrences for DayofMonth and DayOfWeek columns:"))
dayofMonthOccurrencesDF = chicagoDF.groupBy("DayofMonth").agg(count(lit(1)).alias("Total"))
dayOfWeekDF = chicagoDF.groupBy("DayOfWeek").agg(count(lit(1)).alias("Total"))

leastFreqDayOfMonth    = dayofMonthOccurrencesDF.orderBy(col("Total").asc()).first()
mostFreqDayOfMonth     = dayofMonthOccurrencesDF.orderBy(col("Total").desc()).first()
leastFreqDayOfWeek     = dayOfWeekDF.orderBy(col("Total").asc()).first()
mostFreqDayOfWeek      = dayOfWeekDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqDayOfMonth", "mostFreqDayOfMonth", "leastFreqDayOfWeek", "mostFreqDayOfWeek", \
       "%d (%d occurrences)" % (leastFreqDayOfMonth["DayofMonth"], leastFreqDayOfMonth["Total"]), \
       "%d (%d occurrences)" % (mostFreqDayOfMonth["DayofMonth"], mostFreqDayOfMonth["Total"]), \
       "%d (%d occurrences)" % (leastFreqDayOfWeek["DayOfWeek"], leastFreqDayOfWeek["Total"]), \
       "%d (%d occurrences)" % (mostFreqDayOfWeek["DayOfWeek"], mostFreqDayOfWeek["Total"]))))
display(Markdown(f"The most frequent day of the month is day {(mostFreqDayOfMonth['DayofMonth']) }"))
display(Markdown(f"The most frequent day of the week is day {mostFreqDayOfWeek['DayOfWeek']}"))


#Look at extremes for month and year
display(Markdown("Most and least frequent occurrences for Month and Year columns:"))
MonthOccurrencesDF = chicagoDF.groupBy("Month").agg(count(lit(1)).alias("Total"))
YearOccurencesDF = chicagoDF.groupBy("Year").agg(count(lit(1)).alias("Total"))

leastFreqMonth    = MonthOccurrencesDF.orderBy(col("Total").asc()).first()
mostFreqMonth     = MonthOccurrencesDF.orderBy(col("Total").desc()).first()
leastFreqYear     = YearOccurencesDF.orderBy(col("Total").asc()).first()
mostFreqYear      = YearOccurencesDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqMonth", "mostFreqMonth", "leastFreqYear", "mostFreqYear", \
       "%d (%d occurrences)" % (leastFreqMonth["Month"], leastFreqMonth["Total"]), \
       "%d (%d occurrences)" % (mostFreqMonth["Month"], mostFreqMonth["Total"]), \
       "%d (%d occurrences)" % (leastFreqYear["Year"], leastFreqYear["Total"]), \
       "%d (%d occurrences)" % (mostFreqYear["Year"], mostFreqYear["Total"]))))
display(Markdown(f"The most frequent month is  {(mostFreqMonth['Month']) }"))
display(Markdown(f"The most frequent year is {mostFreqYear['Year']}"))

Most and least frequent occurrences for hour_am_pm:


| leastFreqHour | mostFreqHour |
|----|----|
| 5 (2450 occurrences) | 24 (10926 occurrences) |


The most frequent hour is  24

Most and least frequent occurrences for DayofMonth and DayOfWeek columns:


| leastFreqDayOfMonth | mostFreqDayOfMonth | leastFreqDayOfWeek | mostFreqDayOfWeek |
|----|----|----|----|
| 31 (3573 occurrences) | 1 (6977 occurrences) | 7 (25006 occurrences) | 5 (28190 occurrences) |


...

The most frequent day of the week is day 5

Most and least frequent occurrences for Month and Year columns:


| leastFreqMonth | mostFreqMonth | leastFreqYear | mostFreqYear |
|----|----|----|----|
| 2 (11926 occurrences) | 5 (19551 occurrences) | 2005 (44838 occurrences) | 2006 (79275 occurrences) |


The most frequent month is  5

The most frequent year is 2006

In [15]:
display(Markdown(">### C. Crime-specific columns profiling"))

from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

display(Markdown("Columns of interest: IUCR, Primary Type, Description, Arrest, Domestic"))
display(Markdown("Crimes-specific columns are important to understand what type of crimes are more frequents and gain insights related to their nature.\
i.e. which districts / communities are safer; which are unsafe."))

display(Markdown(("Check for missing values in columns  IUCR, Primary Type, Description, Arrest and Domestic:")))
chicagoDF.select([count(when(col(c).isNull(), c)).alias(c) for c in \
                    ["IUCR","Primary Type","Description","Arrest",\
                    "Domestic"]]).show()
display(Markdown(("Number of distinct values in columns  IUCR, Primary Type, Description, Arrest and Domestic:")))
chicagoDF.select([countDistinct(c).alias(c) for c in ["IUCR", "Primary Type", "Description", "Arrest", "Domestic" ]]).show()

>### C. Crime-specific columns profiling

Columns of interest: IUCR, Primary Type, Description, Arrest, Domestic

Crimes-specific columns are important to understand what type of crimes are more frequents and gain insights related to their nature.i.e. which districts / communities are safer; which are unsafe.

Check for missing values in columns  IUCR, Primary Type, Description, Arrest and Domestic:

+----+------------+-----------+------+--------+
|IUCR|Primary Type|Description|Arrest|Domestic|
+----+------------+-----------+------+--------+
|   0|           0|          0|     0|       0|
+----+------------+-----------+------+--------+



Number of distinct values in columns  IUCR, Primary Type, Description, Arrest and Domestic:

+----+------------+-----------+------+--------+
|IUCR|Primary Type|Description|Arrest|Domestic|
+----+------------+-----------+------+--------+
| 316|          29|        296|     2|       2|
+----+------------+-----------+------+--------+



In [16]:
display(Markdown("Most and least frequent occurrences for crime-related columns:"))
IucrDF         = chicagoDF.groupBy("IUCR").agg(count(lit(1)).alias("Total"))
PrimaryTypeDF  = chicagoDF.groupBy("Primary Type").agg(count(lit(1)).alias("Total"))
DesDF          = chicagoDF.groupBy("Description").agg(count(lit(1)).alias("Total"))

leastFreqIUCR    = IucrDF.orderBy(col("Total").asc()).first()
mostFreqIUCR     = IucrDF.orderBy(col("Total").desc()).first()
leastFreqPrimary = PrimaryTypeDF.orderBy(col("Total").asc()).first()
mostFreqPrimary  = PrimaryTypeDF.orderBy(col("Total").desc()).first()
leastFreqDesc    = DesDF.orderBy(col("Total").asc()).first()
mostFreqDesc     = DesDF.orderBy(col("Total").desc()).first()


display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqDescription", "mostFreqDescription", "leastFreqPrimary", "mostFreqPrimary", \
       "%s (%d occurrences)" % (leastFreqDesc["Description"], leastFreqDesc["Total"]), \
       "%s (%d occurrences)" % (mostFreqDesc["Description"], mostFreqDesc["Total"]), \
       "%s (%d occurrences)" % (leastFreqPrimary["Primary Type"], leastFreqPrimary["Total"]), \
       "%s (%d occurrences)" % (mostFreqPrimary["Primary Type"], mostFreqPrimary["Total"]))))

#check for the boolean columns
ArrestDF          = chicagoDF.groupBy("Arrest").agg(count(lit(1)).alias("Total"))
DomesticDF        = chicagoDF.groupBy("Domestic").agg(count(lit(1)).alias("Total"))
leastFreqArrest   = ArrestDF.orderBy(col("Total").asc()).first()
mostFreqArrest    = ArrestDF.orderBy(col("Total").desc()).first()
leastFreqDomestic = DomesticDF.orderBy(col("Total").asc()).first()
mostFreqDomestic  = DomesticDF.orderBy(col("Total").desc()).first()
print(mostFreqArrest)
print(mostFreqDomestic)

Most and least frequent occurrences for crime-related columns:


| leastFreqDescription | mostFreqDescription | leastFreqPrimary | mostFreqPrimary |
|----|----|----|----|
| JUVENILE PIMPING (1 occurrences) | SIMPLE (20726 occurrences) | RITUALISM (1 occurrences) | THEFT (35145 occurrences) |


Row(Arrest=False, Total=128647)
Row(Domestic=False, Total=164598)


## 5. Answer questions to understand crime in the city of Chicago


>### A. Top 5's:



In [17]:
#Primary type and Location of the crime vs. Arrests.
#different kinds of crime: thefts which involve the criminal leaving the scene vs. arrests for drug busts and domestic violence.

from pyspark.sql.functions import concat
display(Markdown("These are the top 5 Location Descriptions when arrests are made:"))
chicagoDF.where("Arrest = true").groupBy("LocationDescription").count().orderBy(col("count").desc()).limit(5).show()
display(Markdown("These are the top 5 Primary Types and their accompanying Location Description:"))
chicagoDF.where("Arrest = true").groupBy("LocationDescription", "Primary Type").count().orderBy(col("count").desc()).limit(5).show()

display(Markdown("These are the top 5 Location Descriptions when no arrest is made:"))
chicagoDF.where("Arrest = false").groupBy("LocationDescription").count().orderBy(col("count").desc()).limit(5).show()
display(Markdown("These are the top 5 Primary Types and accompanying Locations Descriptions:"))
chicagoDF.where("Arrest = false").groupBy("LocationDescription", "Primary Type").count().orderBy(col("count").desc()).limit(5).show()


display(Markdown("Insight: crimes that lead to arrests are usually committed in public places. The top 5 primary types vary:\
                it is harder to make arrests for crimes related to drugs and prostitution where evidene often lacks.\
                On the contrary, it is easier to make arrests related to thefts, if the person is caught with stolen goods"))

These are the top 5 Location Descriptions when arrests are made:

+-------------------+-----+
|LocationDescription|count|
+-------------------+-----+
|             STREET|15904|
|           SIDEWALK|11819|
|          RESIDENCE| 4452|
|          APARTMENT| 3589|
|              ALLEY| 2212|
+-------------------+-----+



These are the top 5 Primary Types and their accompanying Location Description:

+-------------------+------------+-----+
|LocationDescription|Primary Type|count|
+-------------------+------------+-----+
|           SIDEWALK|   NARCOTICS| 8127|
|             STREET|   NARCOTICS| 8117|
|             STREET|PROSTITUTION| 2343|
|          APARTMENT|     BATTERY| 1896|
|          RESIDENCE|     BATTERY| 1654|
+-------------------+------------+-----+



These are the top 5 Location Descriptions when no arrest is made:

+-------------------+-----+
|LocationDescription|count|
+-------------------+-----+
|             STREET|34132|
|          RESIDENCE|25824|
|          APARTMENT|14317|
|           SIDEWALK| 8931|
|              OTHER| 5242|
+-------------------+-----+



These are the top 5 Primary Types and accompanying Locations Descriptions:

+-------------------+-------------------+-----+
|LocationDescription|       Primary Type|count|
+-------------------+-------------------+-----+
|             STREET|              THEFT| 9648|
|             STREET|    CRIMINAL DAMAGE| 7290|
|             STREET|MOTOR VEHICLE THEFT| 6282|
|          RESIDENCE|      OTHER OFFENSE| 5308|
|          RESIDENCE|            BATTERY| 5173|
+-------------------+-------------------+-----+



Insight: crimes that lead to arrests are usually committed in public places. The top 5 primary types vary:                it is harder to make arrests for crimes related to drugs and prostitution where evidene often lacks.                On the contrary, it is easier to make arrests related to thefts, if the person is caught with stolen goods

>### B. Ratio of number of crimes & frequency of crime type by day time category

In [18]:
from pyspark.sql.functions import count, round
# ratio of crimes primary description by time of day

# Time of day categorized as follows:
#   "morning"   - hour_am_pm= [6,11]
#   "day-time"  - hour_am_pm= [12,18]
#   "evening"   - hour_am_pm= [18,24] 
#   "night"     - hour_am_pm= [1,5]

totalCrimes = chicagoDF.count()

ChicagoDayCategoriesDF = chicagoDF\
   .withColumn("TimeDayCategory", when((col("hour_am_pm")>=6) & (col("hour_am_pm") <12),"1.morning")\
                               .when((col("hour_am_pm")>=12) & (col("hour_am_pm")<18),"2.day")\
                               .when((col("hour_am_pm")>=18) & (col("hour_am_pm")<24),"3.evening")\
                               .otherwise("4.night"))
ChicagoDayCategoriesDF.cache()
#ChicagoDayCategoriesDF.select("hour_am_pm","TimeDayCategory").show(8)

from pyspark.sql.functions import count, round

display(Markdown("Breakdown of the number of crimes committed by TimeDayCategory"))
ChicagoDayCategoriesDF.select("TimeDayCategory")\
                      .groupBy("TimeDayCategory")\
                      .agg(count("TimeDayCategory").alias("Count"),\
                            (count("TimeDayCategory")/totalCrimes*100).alias("Ratio"))\
                      .orderBy(col("Count").desc())\
                      .select("TimeDayCategory", "Count",round("Ratio",2).alias("RoundedRatio")).show()

display(Markdown("Most crimes are committed in the evening."))

display(Markdown("Types of crimes (Description) that occur at different moments of the day:"))

display(Markdown("Morning:"))
ChicagoDayCategoriesDF.where("(TimeDayCategory = '1.morning')").select("Primary Type", "Description","TimeDayCategory")\
                    .groupBy("Primary Type", "Description")\
                    .count().orderBy(col("count").desc())\
                    .limit(5)\
                    .show()

display(Markdown("Day time:"))
ChicagoDayCategoriesDF.where("(TimeDayCategory = '2.day')").select("Primary Type", "Description","TimeDayCategory")\
                    .groupBy("Primary Type", "Description")\
                    .count().orderBy(col("count").desc())\
                    .limit(5)\
                    .show()

display(Markdown("Evening:"))
ChicagoDayCategoriesDF.where("(TimeDayCategory = '3.evening')").select("Primary Type", "Description","TimeDayCategory")\
                    .groupBy("Primary Type", "Description")\
                    .count().orderBy(col("count").desc())\
                    .limit(5)\
                    .show()

display(Markdown("Night:"))
ChicagoDayCategoriesDF.where("(TimeDayCategory = '4.night')").select("Primary Type", "Description","TimeDayCategory")\
                    .groupBy("Primary Type", "Description")\
                    .count().orderBy(col("count").desc())\
                    .limit(5)\
                    .show()

Breakdown of the number of crimes committed by TimeDayCategory

+---------------+-----+------------+
|TimeDayCategory|Count|RoundedRatio|
+---------------+-----+------------+
|      3.evening|61482|       33.06|
|          2.day|56140|       30.19|
|      1.morning|36425|       19.59|
|        4.night|31912|       17.16|
+---------------+-----+------------+



Most crimes are committed in the evening.

Types of crimes (Description) that occur at different moments of the day:

Morning:

...

+------------+--------------------+-----+
|Primary Type|         Description|count|
+------------+--------------------+-----+
|     BATTERY|              SIMPLE| 4567|
|     BATTERY|DOMESTIC BATTERY ...| 3940|
|       THEFT|      $500 AND UNDER| 3289|
|     ASSAULT|              SIMPLE| 2989|
|       THEFT|       FROM BUILDING| 2805|
+------------+--------------------+-----+



Evening:

+---------------+--------------------+-----+
|   Primary Type|         Description|count|
+---------------+--------------------+-----+
|        BATTERY|DOMESTIC BATTERY ...| 5198|
|      NARCOTICS|POSS: CANNABIS 30...| 4545|
|        BATTERY|              SIMPLE| 4104|
|CRIMINAL DAMAGE|          TO VEHICLE| 4063|
|          THEFT|      $500 AND UNDER| 3598|
+---------------+--------------------+-----+



Night:

+---------------+--------------------+-----+
|   Primary Type|         Description|count|
+---------------+--------------------+-----+
|        BATTERY|DOMESTIC BATTERY ...| 3778|
|CRIMINAL DAMAGE|          TO VEHICLE| 2460|
|CRIMINAL DAMAGE|         TO PROPERTY| 2375|
|        BATTERY|              SIMPLE| 2229|
|          THEFT|      $500 AND UNDER| 1856|
+---------------+--------------------+-----+



>### C. Ratios of crimes types: week vs. weekend

In [19]:
from pyspark.sql.functions import count, round

# Crime: week vs. weekend categorized as follows:
#   "Week"    - DayOfWeek =[1,5] 
#   "Weekend" - DayOfWeek =[6,7] 

TotalCrime = chicagoDF.count()
WeekDF = chicagoDF\
   .withColumn("WeekCategory", when(col("DayOfWeek")<6,"Week")\
                               .otherwise("Weekend"))

TotalCrimeWeek = WeekDF.where(col("WeekCategory")=="Week").count()
TotalCrimeWeekend = WeekDF.where(col("WeekCategory")=="Weekend").count()

display(Markdown("Crimes committed during the week by ratio: top 5"))
WeekDF.select("WeekCategory", "Primary Type")\
                    .where(F.col("WeekCategory") == 'Week')\
                      .groupBy("WeekCategory","Primary Type")\
                      .agg(count("WeekCategory").alias("Count"),\
                            (count("WeekCategory")/totalCrimes*100).alias("Ratio"))\
                      .orderBy(col("Count").desc())\
                      .select("WeekCategory","Primary Type", "Count",round("Ratio",2).alias("RoundedRatio")).limit(5).show()

display(Markdown("Crimes committed during the weekend by ratio: top 5"))
WeekDF.select("WeekCategory", "Primary Type")\
                    .where(F.col("WeekCategory") == 'Weekend')\
                      .groupBy("WeekCategory","Primary Type")\
                      .agg(count("WeekCategory").alias("Count"),\
                            (count("WeekCategory")/totalCrimes*100).alias("Ratio"))\
                      .orderBy(col("Count").desc())\
                      .select("WeekCategory","Primary Type", "Count",round("Ratio",2).alias("RoundedRatio")).limit(5).show()

display(Markdown("Roughly the same type of offense are committed throughout the week and weekend: we have the same top 5 but in different order."))

display(Markdown("For more detail on the crimes committed we can look at the pivot for Primary Type:"))
pivot = WeekDF.groupBy("Primary Type").pivot("WeekCategory").agg(F.count("Primary Type"))
pivot.show()


WeekDF.cache() # optimization to make the processing faster

#check that there are two new categories
display(Markdown("Check that there are 2 categories:"))
WeekDF.select([countDistinct(c).alias(c) for c in ["WeekCategory"]]).show()

#distribution of crimes week vs. weekend (top 10 Primary Type)
display(Markdown("Look at the distribution of crime types: week versus weekend, here the ratios are based \
not on total number of crimes, but total number of crimes by WeekCategory (week or weekend)"))


display(Markdown("Top 10 most common crime types and frequency: Week days"))
display(Markdown(f"The total number of crime occurrences during the week is {TotalCrimeWeek} crimes"))
WeekDF.where(col("WeekCategory")=="Week")\
                     .select("Primary Type", "WeekCategory")\
                     .groupBy("Primary Type")\
                     .agg(count("Primary Type").alias("FreqPrimaryType"), \
                          (count("Primary Type")/TotalCrimeWeek*100).alias("Ratio"))\
                     .orderBy(col("Ratio").desc())\
                     .select("Primary Type","FreqPrimaryType",round("Ratio",2).alias("RoundedRatio")).limit(15).show()

display(Markdown("Top 10 most common crime types and frequency: Weekends"))
display(Markdown(f"The total number of crime occurrences during the weekend is {TotalCrimeWeekend} crimes"))

WeekDF.where(col("WeekCategory")=="Weekend")\
                     .select("Primary Type", "WeekCategory")\
                     .groupBy("Primary Type")\
                     .agg(count("Primary Type").alias("FreqPrimaryType"), \
                          (count("Primary Type")/TotalCrimeWeekend*100).alias("Ratio"))\
                     .orderBy(col("Ratio").desc())\
                     .select("Primary Type","FreqPrimaryType",round("Ratio",2).alias("RoundedRatio")).limit(15).show()


Crimes committed during the week by ratio: top 5

+------------+---------------+-----+------------+
|WeekCategory|   Primary Type|Count|RoundedRatio|
+------------+---------------+-----+------------+
|        Week|          THEFT|25951|       13.96|
|        Week|        BATTERY|23229|       12.49|
|        Week|      NARCOTICS|17478|         9.4|
|        Week|CRIMINAL DAMAGE|15909|        8.56|
|        Week|  OTHER OFFENSE| 8254|        4.44|
+------------+---------------+-----+------------+



Crimes committed during the weekend by ratio: top 5

+------------+---------------+-----+------------+
|WeekCategory|   Primary Type|Count|RoundedRatio|
+------------+---------------+-----+------------+
|     Weekend|        BATTERY|10761|        5.79|
|     Weekend|          THEFT| 9194|        4.94|
|     Weekend|CRIMINAL DAMAGE| 7097|        3.82|
|     Weekend|      NARCOTICS| 6195|        3.33|
|     Weekend|  OTHER OFFENSE| 3209|        1.73|
+------------+---------------+-----+------------+



Roughly the same type of offense are committed throughout the week and weekend: we have the same top 5 but in different order.

...

The total number of crime occurrences during the week is 134330 crimes

+--------------------+---------------+------------+
|        Primary Type|FreqPrimaryType|RoundedRatio|
+--------------------+---------------+------------+
|               THEFT|          25951|       19.32|
|             BATTERY|          23229|       17.29|
|           NARCOTICS|          17478|       13.01|
|     CRIMINAL DAMAGE|          15909|       11.84|
|       OTHER OFFENSE|           8254|        6.14|
|             ASSAULT|           8235|        6.13|
|            BURGLARY|           7884|        5.87|
| MOTOR VEHICLE THEFT|           6202|        4.62|
|             ROBBERY|           4712|        3.51|
|   CRIMINAL TRESPASS|           4556|        3.39|
|  DECEPTIVE PRACTICE|           4471|        3.33|
|        PROSTITUTION|           2258|        1.68|
|   WEAPONS VIOLATION|           1204|         0.9|
|PUBLIC PEACE VIOL...|            929|        0.69|
|OFFENSE INVOLVING...|            761|        0.57|
+--------------------+---------------+------------+



Top 10 most common crime types and frequency: Weekends

The total number of crime occurrences during the weekend is 51629 crimes

+--------------------+---------------+------------+
|        Primary Type|FreqPrimaryType|RoundedRatio|
+--------------------+---------------+------------+
|             BATTERY|          10761|       20.84|
|               THEFT|           9194|       17.81|
|     CRIMINAL DAMAGE|           7097|       13.75|
|           NARCOTICS|           6195|        12.0|
|       OTHER OFFENSE|           3209|        6.22|
|             ASSAULT|           2989|        5.79|
|            BURGLARY|           2423|        4.69|
| MOTOR VEHICLE THEFT|           2373|         4.6|
|             ROBBERY|           1948|        3.77|
|   CRIMINAL TRESPASS|           1561|        3.02|
|  DECEPTIVE PRACTICE|           1205|        2.33|
|        PROSTITUTION|            566|         1.1|
|   WEAPONS VIOLATION|            422|        0.82|
|OFFENSE INVOLVING...|            367|        0.71|
|PUBLIC PEACE VIOL...|            335|        0.65|
+--------------------+---------------+------------+



In [20]:
display(Markdown(">### D. Safety in the districts"))

pivotDistrict = chicagoDF.groupBy("District").pivot("DayOfWeek").agg(F.count("Primary Type").alias("Count"))

display(Markdown("The pivot table below shows number of crime by day of the week for each district:"))

pivotDistrict.limit(25).show(25)

TotalCrimes = chicagoDF.count()
District_arrestDF = chicagoDF\
    .withColumn("Escalate_category", when((col("Arrest")== True) & (col("Domestic") == True), "1. Domestic Arrest")\
                            .when((col("Arrest") == True) & (col("Domestic") == False), "2. General Arrest")\
                            .when((col("Arrest") == False) & (col("Domestic") == True), "3. Domestic Violence")\
                            .when((col("Arrest") == False) & (col("Domestic") == False), "4. General Violence"))
                
District_arrestDF.cache()

display(Markdown("The table below shows ratio by Escalate_category:"))

totalCrimes = chicagoDF.count()
District_arrestDF.select("Escalate_category")\
                      .groupBy("Escalate_category")\
                      .agg(count("Escalate_category").alias("Count"),\
                            (count("Escalate_category")/totalCrimes*100).alias("Ratio"))\
                      .orderBy(col("Count").desc())\
                      .select("Escalate_category", round("Ratio",2)).show()

display(Markdown("The majority of crimes are related to general offenses that do not result in an arrest."))

display(Markdown("The table below is a break down by district and escalate type:"))

District_arrestDF.select("Escalate_category","District")\
                      .groupBy("Escalate_category","District")\
                      .agg(count("Escalate_category").alias("Count"),\
                            (count("Escalate_category")/totalCrimes*100).alias("Ratio"))\
                      .orderBy(col("Count").desc())\
                      .select("Escalate_category","District", round("Ratio",2)).show()

pivot_district_by_escalate = District_arrestDF.groupBy("District").pivot("Escalate_category").agg(F.count("District"))

pivot_district_by_escalate.show()



>### D. Safety in the districts

The pivot table below shows number of crime by day of the week for each district:

+--------+----+----+----+----+----+----+----+
|District|   1|   2|   3|   4|   5|   6|   7|
+--------+----+----+----+----+----+----+----+
|      31|   1|null|null|null|null|   1|null|
|      12|1277|1267|1254|1379|1466|1224|1213|
|      22| 868| 862| 978| 924| 973| 853| 844|
|       1| 922|1053|1087|1037|1120| 950| 734|
|      16| 838| 764| 878| 849| 893| 842| 917|
|       6|1479|1555|1599|1528|1551|1491|1441|
|       3|1362|1481|1390|1440|1454|1412|1325|
|      20| 456| 482| 503| 493| 498| 487| 448|
|       5|1212|1255|1215|1229|1269|1140|1068|
|      19|1086|1056|1048|1110|1195|1229|1108|
|      15|1100|1242|1256|1213|1278|1281|1181|
|      17| 779| 771| 762| 747| 845| 723| 715|
|       9|1366|1356|1295|1377|1410|1389|1363|
|       4|1494|1596|1608|1618|1676|1486|1412|
|       8|1864|1864|1904|1931|2009|1880|1887|
|       7|1610|1639|1656|1716|1697|1674|1538|
|      10|1052|1105|1092|1074|1148|1122|1061|
|      25|1548|1417|1447|1443|1534|1526|1393|
|      24| 840| 863| 797| 802| 869

The table below shows ratio by Escalate_category:

+--------------------+---------------+
|   Escalate_category|round(Ratio, 2)|
+--------------------+---------------+
| 4. General Violence|          60.15|
|   2. General Arrest|          28.36|
|3. Domestic Violence|           9.03|
|  1. Domestic Arrest|           2.46|
+--------------------+---------------+



The majority of crimes are related to general offenses that do not result in an arrest.

The table below is a break down by district and escalate type:

+-------------------+--------+---------------+
|  Escalate_category|District|round(Ratio, 2)|
+-------------------+--------+---------------+
|4. General Violence|       8|            4.5|
|4. General Violence|       4|           3.97|
|4. General Violence|       7|           3.39|
|4. General Violence|       6|           3.38|
|4. General Violence|      12|           3.22|
|4. General Violence|      25|           3.21|
|4. General Violence|       9|           3.02|
|4. General Violence|       3|           2.89|
|4. General Violence|      19|           2.88|
|4. General Violence|       2|           2.77|
|4. General Violence|      18|           2.76|
|4. General Violence|       5|           2.76|
|4. General Violence|      11|           2.75|
|  2. General Arrest|      11|           2.75|
|4. General Violence|      14|           2.53|
|4. General Violence|      16|           2.31|
|4. General Violence|      10|           2.27|
|4. General Violence|       1|           2.23|
|4. General V